In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import shutil
import cv2
from tqdm import tqdm

from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from keras.layers import Add, Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D , Dropout
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

from keras.callbacks import ModelCheckpoint
from keras.initializers import glorot_uniform

from keras.applications.vgg16 import VGG16

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Declare constants
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
BATCH_SIZE = 64
N_CHANNELS = 3
N_CLASSES = 20
IMG_SHAPE = (IMAGE_HEIGHT,IMAGE_WIDTH,N_CHANNELS)

AUD_HEIGHT = 128
AUD_WIDTH = 128


In [ ]:
# Creating training data
# dont run this
main_dir = '/content/gdrive/My Drive'
data_dir = main_dir + '/faces20/'
categories = os.listdir(main_dir + '/faces20')
data = []
categorical_index = []
def create_face_data():
  for category in categories:
    path = os.path.join(data_dir+category)
    class_num = categories.index(category)
    categorical_index.append([category,class_num])
    for img in tqdm(os.listdir(path)):
      try:
        img_array_bgr = cv2.imread(os.path.join(path,img))
        img_array = cv2.cvtColor(img_array_bgr , cv2.COLOR_BGR2RGB)
        new_array = cv2.resize(img_array , (IMAGE_HEIGHT,IMAGE_WIDTH))
        data.append([new_array,class_num])
      except Exception as e:
        pass

create_face_data()

100%|██████████| 39/39 [00:14<00:00,  2.61it/s]


In [ ]:
len(data)

1191

In [ ]:
main_dir1 = '/content/gdrive/My Drive'
data_dir1 = main_dir1 + '/t3/audio-images/'
categories1 = os.listdir(main_dir1 + '/t3/audio-images')
data1 = []
categorical_index1 = []
def create_audio_data():
  for category in categories1:
    path = os.path.join(data_dir1+category)
    class_num = categories1.index(category)
    categorical_index1.append([category,class_num])
    for img in tqdm(os.listdir(path)):
      try:
        img_array_bgr = cv2.imread(os.path.join(path,img))
        img_array = cv2.cvtColor(img_array_bgr , cv2.COLOR_BGR2RGB)
        new_array = cv2.resize(img_array , (IMAGE_HEIGHT,IMAGE_WIDTH))
        data1.append([new_array,class_num])
      except Exception as e:
        pass

create_audio_data()

100%|██████████| 163/163 [00:01<00:00, 127.04it/s]


In [ ]:
len(data1)

4177

In [ ]:
# dont run this
import random

random.shuffle(data)

In [ ]:
# creating X and y from data list

X = []
y = []

for features, labels in data:
  X.append(features)
  y.append(labels)

# converting into numpy arrays
X = np.array(X).reshape(-1,IMAGE_HEIGHT,IMAGE_WIDTH,3)
y = np.array(y)

In [ ]:
random.shuffle(data1)

In [ ]:
# creating X and y from data list

X1 = []
y1 = []

for features, labels in data1:
  X1.append(features)
  y1.append(labels)

# converting into numpy arrays
X1 = np.array(X1).reshape(-1,IMAGE_HEIGHT,IMAGE_WIDTH,3)
y1 = np.array(y1)

In [ ]:
# Save the created data using pickle so we don't have tp run the create_data() function again

import pickle

pickle_out = open('/content/gdrive/My Drive/data/X.pickle' , 'wb')
pickle.dump(X , pickle_out)
pickle_out.close()

pickle_out = open('/content/gdrive/My Drive/data/y.pickle' , 'wb')
pickle.dump(y , pickle_out)
pickle_out.close()

In [ ]:
import pickle

pickle_out = open('/content/gdrive/My Drive/data/X1.pickle' , 'wb')
pickle.dump(X1 , pickle_out)
pickle_out.close()

pickle_out = open('/content/gdrive/My Drive/data/y1.pickle' , 'wb')
pickle.dump(y1 , pickle_out)
pickle_out.close()

In [ ]:
# load the saved data

# start running from here

pickle_in = open('/content/gdrive/My Drive/data/X.pickle' , 'rb')
X = pickle.load(pickle_in)

In [ ]:
# load audio saved data

pickle_in = open('/content/gdrive/My Drive/data/X1.pickle' , 'rb')
X1 = pickle.load(pickle_in)

pickle_in = open('/content/gdrive/My Drive/data/y1.pickle' , 'rb')
y1 = pickle.load(pickle_in)

In [ ]:
# creating train test split data

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)

print('Shape of X_train:' + str(X_train.shape))
print('Shape of y_train:' + str(y_train.shape))
print('Shape of X_test:' + str(X_test.shape))
print('Shape of y_test:' + str(y_test.shape))

Shape of X_train:(1071, 128, 128, 3)
Shape of y_train:(1071,)
Shape of X_test:(120, 128, 128, 3)
Shape of y_test:(120,)


In [ ]:
# creating train test split for audio data

from sklearn.model_selection import train_test_split

X1_train,X1_test,y1_train,y1_test = train_test_split(X1,y1,test_size=0.1)

print('Shape of X_train:' + str(X1_train.shape))
print('Shape of y_train:' + str(y1_train.shape))
print('Shape of X_test:' + str(X1_test.shape))
print('Shape of y_test:' + str(y1_test.shape))

Shape of X_train:(3759, 128, 128, 3)
Shape of y_train:(3759,)
Shape of X_test:(418, 128, 128, 3)
Shape of y_test:(418,)


In [ ]:
img_a = []
img_b = []
img_y = []

def create_image_data_pairs():

  # our objective is to make half of the dataset witht the same classes and half with different classes.

  # same class
  i = 0
  while(i<2500):

    # perform pairing for same classes.
    same_class = np.random.randint(N_CLASSES) # 5

    index = random.choice([k for k,j in enumerate(y) if j==same_class] , 2, replace=False)
    x_random = X[index]
    y_same = same_class

    img_a.append(x_random[0])
    img_b.append(x_random[1])

    img_y.append(y_same)

  # different classes
  i = 0
  while(i<2500):
    # Repeat this as many times as many examples you want using a loop

    # select random indices from the shape[0] of the X_train array
    index = np.random.choice(X.shape[0], 2, replace=False)  

    # make a random array by choosing subsets from X_train
    x_random = X[index]
    y_random = y[index]

    # append the first image to img_a and second to img_b
    img_a.append(x_random[0])
    img_b.append(x_random[1])

    # finally based on the similarity of the images based on their classes append either the class (if they are similar) or -1 if they are dissimilar
    if(y_random[0] == y_random[1]):
      pass
    else:
      img_y.append(-1)
      i = i+1
    

  return img_a,img_b,img_y


In [ ]:

aud_a = []
aud_b = []
aud_y = []

def create_audio_data_pairs():

  # our objective is to make half of the dataset witht the same classes and half with different classes.
  i = 0

  while(i<2500):

    # perform pairing for same classes.
    same_class = np.random.randint(N_CLASSES)
    index = random.choice([k for k,j in enumerate(y) if j==same_class] , 2, replace=False)
    x_random = X1[index]
    y_same = same_class

    aud_a.append(x_random[0])
    aud_b.append(x_random[1])

    aud_y.append(y_same)

  i = 0

  # different classes
  while(i < 2500):

    index = np.random.choice(X1.shape[0] , 2 ,replace = False)

    x1_random = X1[index]
    y1_random = y1[index]

    # append the first audio to aud_a and second to aud_b
    aud_a.append(x1_random[0])
    aud_b.append(x1_random[1])

    # finally based on the similarity of the audio images based on their classes append either the class (if they are similar) or -1 if they are dissimilar
    if(y1_random[0] == y1_random[1]):
      pass
    else:
      aud_y.append(-1)
      i = i+1
    

    return aud_a , aud_b , aud_y

In [ ]:
# create the final data to be fed into the siamese network for training

def multimodal_data_generate():
  img_a , img_b , img_y = create_image_data_pairs()
  aud_a , aud_b , aud_y = create_audio_data_pairs()

  labels = []

  for i in range(len(img_a)):
    if(img_y==-1 or aud_y == -1):
      labels.append(0)
    elif(img_y==aud_y):
      labels.append(1)
    else:
      labels.append(0)
      
 return img_a,aud_a,img_b,aud_b,labels   

In [ ]:
from keras.applications.vgg16 import VGG16
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False
base_model.summary()

inputs = Input(shape=IMG_SHAPE)
  x = inputs
  x = base_model(x, training=False)
  x = Flatten()(x)
  x = Dense(128, activation='relu',kernel_regularizer='l2')(x)
  x = Dropout(0.5)(x)
  outputs = Dense(N_CLASSES , activation='softmax')(x)

  model = Model(inputs,outputs)
  model.summary()

In [ ]:
# create the individual networks for image and audio to get the feature encodings
def img_network(input_dim_img):
  base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

  base_model.trainable = False
  base_model.summary()

  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Input(shape=IMG_SHAPE))
  model.add(base_model)
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(128, activation='relu',kernel_regularizer='l2'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.summary()

  return model



def aud_network(input_dim_aud):

  IMAGE_HEIGHT , IMAGE_WIDTH , N_CHANNELS = input_dim_aud # input shape

  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
  model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
  #model.add(tf.keras.layers.Dropout(0.005))
  #model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  #model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
  #model.add(tf.keras.layers.BatchNormalization())
  #model.add(tf.keras.layers.Dropout(0.01))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  #model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
  #model.add(tf.keras.layers.BatchNormalization())
  #model.add(tf.keras.layers.Dropout(0.05))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  #model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256, activation='relu'))
  #model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(64, activation='relu'))
  #model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(0.25))

  return model

  
def multimodal_network(input_dim):
  pass

In [ ]:
# siamese network model which helps in one-shot learning

def siamese_network(input_dim_img,input_dim_aud):
    
    img_a = Input(shape=input_dim_img)
    img_b = Input(shape=input_dim_img)
    aud_a = Input(shape=input_dim_aud)
    aud_b = Input(shape=input_dim_aud)

    # extracting the image embeddings from the image network for two input images


    img_network = image_feat_network(input_dim_img)

    # img_network.load_weights()

    feat_img_a = img_network(img_a)
    feat_img_b = img_network(img_b)
    
    # extracting the audio embeddings from the image network for two output images

    aud_network = audio_feat_network(input_dim_aud)

    # aud_network.load_weights()

    feat_aud_a = aud_network(aud_a)
    feat_aud_b = aud_network(aud_b)
    
    # concatenate the first image's embeddings with the first audio's embeddings
    concat_a = Concatenate(axis=3)([feat_img_a, feat_aud_a])
    # concatenate the second image's embeddings with the second audio's embeddings
    concat_b = Concatenate(axis=3)([feat_img_b, feat_aud_b])
    
    input_dim = concat_a.shape # (?) put the input dimensions here
    base_network = multi_modal_network(input_dim)

    # left vector encodings
    encoded_l = base_network(concat_a) 
    # right vector encodings
    encoded_r = base_network(concat_b)

    #---->Layer to merge two encoded inputs with the l1 distance between them
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))

    #---->Get the L1 Distance between the two encoded legs and predict a sigmoid using that.
    L1_distance = L1_layer([encoded_l, encoded_r])
    prediction = Dense(1,activation='sigmoid')(L1_distance)

    model = Model(input=[img_a, aud_a, img_b,aud_b], output=prediction)

    optimizer = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999)
    #  model.compile(loss="binary_crossentropy",optimizer=optimizer)

    return optimizer,model

In [ ]:
# final model train 

epochs = 30
input_dim_img = (IMAGE_HEIGHT,IMAGE_WIDTH,N_CHANNELS)
input_dim_aud = (AUD_HEIGHT,AUD_WIDTH,N_CHANNELS)

img_a , aud_a , img_b , aud_b , labels = multimodal_data_generate()
optimizer , model = siamese_network(input_dim_img , input_dim_aud)
model.compile(loss = 'binary_crossentropy' , optimizer=optimizer)
model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
model.fit([img_a,aud_a,img_b,aud_b], labels , validation_fit = 0.25 , batch_size = BATCH_SIZE , verbose = 2, nb_epochs = epochs , callbacks = [es] , metrics = ['accuracy'])

In [ ]:
tf.reset_default_graph()
faceModel = get_face_model()
faceModel.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# cp_callback = ModelCheckpoint('/content/gdrive/My Drive/Face Detection/training4',save_weights_only=True,verbose=1)

weight_dir = "/content/gdrive/My Drive/Face Detection/weight_face_detection_model"
if not os.path.exists(weight_dir):
    os.mkdir(weight_dir)
    
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=weight_dir+'/checkpoint-{epoch:02d}.hdf5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
if os.path.exists(weight_dir):
  faceModel.load_weights(filepath=weight_dir+'/checkpoint-{epoch:02d}.hdf5')

In [ ]:
# Train model for 10 epochs, capture the history
history = faceModel.fit(train_dataset, epochs=10, validation_data=valid_dataset, callbacks=[cp_callback])

In [ ]:
faceModel.save('/content/gdrive/My Drive/models/face model2')

In [ ]:
final_loss, final_acc = faceModel.evaluate(valid_dataset)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()